This notebook aims at applying TiSSNet (T-waves detector) on some acoustic data.

In [ ]:
import datetime
import numpy as np
import torch
from torchvision.transforms import Resize
from matplotlib import pyplot as plt

from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.signal.make_spectrogram import make_spectrogram
from utils.detection.TiSSNet import TiSSNet

In [ ]:
catalog_path = "../../../data/demo"
tissnet_checkpoint = "../../../data/models/TiSSNet/torch_save"

stations = StationsCatalog(catalog_path)
manager = stations.by_name("MAHY01")[0].get_manager()

model_det = TiSSNet()
model_det.load_state_dict(torch.load(tissnet_checkpoint))

In [ ]:
date_start = manager.dataset_start + datetime.timedelta(seconds=150)
date_end = date_start + datetime.timedelta(seconds=200)
data = manager.get_segment(date_start, date_end)

# these configurations are the ones needed by TiSSNet to work (this corresponds to 256 fft points at 240 Hz)
spectrogram = make_spectrogram(data, manager.sampling_f, t_res=0.5342, f_res=0.9375, return_bins=False, normalize=True, vmin=-35, vmax=140).astype(np.float32)
spectrogram = spectrogram[np.newaxis, :, :]  # add a dummy dimension, this stands for the channel number (here we are in grayscale, i.e. only one value for each pixel)
input_data = Resize((128, spectrogram.shape[-1]))(torch.from_numpy(spectrogram))# resize data

with torch.no_grad():  # tells PyTorch that no gradient back propagation is needed (we do not train any network here)
    res = model_det(input_data).numpy()

f = plt.figure(1)
plt.imshow(input_data[0], aspect="auto", cmap="jet", vmin=0, vmax=1)
plt.title("Input spectrogram")
plt.xlabel("Time (spectrogram bins)")
plt.ylabel("Frequency (spectrogram bins)")
f.show()

g = plt.figure(2)
plt.plot(res)
plt.xlim(0, len(res))
plt.ylim(0, 1)
plt.title("TiSSNet result")
plt.xlabel("Time (spectrogram bins)")
plt.ylabel("Estimated probability")
g.show()